In [ ]:
import numpy as np
import math
from sklearn.metrics import accuracy_score 
from tqdm import tqdm
from tensorflow import keras
import tensorflow as tf
from keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [ ]:
x_train = x_train.astype('float128')
x_train = x_train/255
x_test = x_test.astype('float128')
x_test = x_test/255

In [ ]:
types = ['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']
set_data = [0,1,2,3,4,5,6]


# data_dict = {}

# for i in range(7):
#     data_dict[types[i]] = set_data[i]

# print(type(data_dict))
# print(data_dict['batches.meta'])

#Alternate way
batch_meta = set_data[0]
data_batch1 = set_data[1]
data_batch2 = set_data[2]
data_batch3 = set_data[3]
data_batch4 = set_data[4]
data_batch5 = set_data[5]
test_batch = set_data[6]



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils


In [ ]:
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 
num_classes = y_test.shape[1]


In [ ]:
print(y_train.shape)

(50000, 10)


In [ ]:
#Testing plt 
#plt.imshow(x_train[54])

In [ ]:
#One hot encoding function=

def one_hot(vec, vals = 10):
    n = len(vec)
    out = np.zeros((n,vals))
    out[range(n), vec] = 1
    return out

In [ ]:
class CifarClass():    #Creating the class for the relevant methods

    def __init__(self, learning_rate = 0.1, max_iter = 1000, minimum_step_size = 0.001, early_stopping = False, n=20):
        self.i = 0
        #self.training_sets = [data_batch1, data_batch2, data_batch3, data_batch4, data_batch5]
        #self.test_set = [test_batch]
        self.training_images = None
        self.training_labels = None 
        self.testing_images = None
        self.testing_labels = None
        self.__learning_rate = learning_rate
        self.__max_iter = max_iter
        self.__minimum_step_size = minimum_step_size
        self.__early_stopping = early_stopping
        self.__n = n


    def setup_of_images(self):
        self.training_images = x_train
        train_len = len(self.training_images)
        self.training_labels = y_train

        self.testing_images = x_test
        test_len = len(self.testing_images)
        self.testing_labels = y_test

    #calculating the score for x for a given class k
    def __calculate_score(self, k, x):
        weight = self.__weights[k]
        return np.exp(x.dot(weight))

    def __calculate_softmax(self, k, x):
        summation = 0
        for i in range(self.__class_count):
            summation += self.__calculate_score(i,x)

        #print((self.__calculate_score(k,x)/summation).shape)  
        return self.__calculate_score(k,x)/summation

    def __cross_entropy_gradient(self, k):
        sum = 0
        #iterating over all the images in the set
        for i in range (len(self.__x)):
            #print(self.__y[i][k])
            sum += ((self.__calculate_softmax(k,self.__x[i]) - self.__y[i][k]) * self.__x[i]) #the gradient formula
        
        #print(sum)
        return sum / len(self.__x) #returning the average of the all the difference for each image
    
    def __new_weights(self, k): 
        #no change if class is in exclusion list
        if k in self.__exclusion_list:
            return self.__weights[k]

        #print('loss =', math.log(self.__calculate_softmax[k]))

        step_size = self.__cross_entropy_gradient(k)*self.__learning_rate

        #code to add classes to excleusion list
        # if np.amax(step_size) <= self.__minimum_step_size:
        #     self.__exclusion_list.append(k)
        
        return self.__weights[k] - step_size
    
    def train(self, X, y):
        
        # if self.__early_stopping:
        #     split_index = int(len(X) * 0.1)
        #     self.__x = X[split_index:]
        #     self.__y = y[split_index:]
        #     self.__x_val = X[:split_index]
        #     self.__y_val = y[:split_index]
        # else:
        #     self.__x = X
        #     self.__y = y 

        self.__class_count = 10
        self.__weights = np.random.rand(self.__class_count, X.shape[2]) #initialize random weights
        self.__exclusion_list = []
        max_acc = -1
        n=0
        print('initial weight:', cc.__weights)

        for j in tqdm(range(1)):                                            #set number of epochs
            for i in range(3125):                                           #iterating over all the batches 
                if self.__early_stopping:                                   #ignore the if, move to the else
                    split_index = int(len(X) * 0.1)
                    self.__x = X[split_index:]
                    self.__y = y[split_index:]
                    self.__x_val = X[:split_index]
                    self.__y_val = y[:split_index]
                else:
                    self.__x = X[i]                      #passes the i th batch, i is the batch index
                    self.__y = y[i]
                    

                for j in range(self.__class_count):                #sets new weighs for every class index for each batch
                    self.__weights[j] = self.__new_weights(j)
        
        print('final weight:', self.__weights)
        # for j in range(self.__class_count):
        #     self.__weights[j] = self.__new_weights(j)
            
            #stop training once all classes have been added to the exclusion list
        # if len(self.__exclusion_list) == self.__class_count:
        #     break
            
            #if early stopping is enabled
        # if (self.__early_stopping):
        #     y_val_pred = self.predict(self.__x_val)
        #     acc = accuracy_score(self.__y_val, y_val_pred)
        #     if acc == max_acc:
        #         max_acc = acc
        #         n = 0
        #     else:
        #         n += 1
        #         if n >= self.__n:
        #             break
    
    #returns the prediccted label in one hot encoded form
    def predict(self, X):
        y= np.zeros((len(X), self.__class_count))
        for i in tqdm(range(len(X))):
            max_score_idx = 0
            max_score = 0
            for j in range(self.__class_count):
                score = self.__calculate_softmax(j, X[i])
                # print('score', j, 'is', score)
                if score > max_score:
                    max_score = score
                    max_score_idx = j
                    
            # print('loop over')        
            y[i][max_score_idx] = 1
        return y


In [ ]:
cc = CifarClass() 
cc.setup_of_images()

print (cc.training_images.shape)
print(cc.testing_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
#flattening

cc.training_images = cc.training_images.reshape(50000, 3072)

#max_img = np.amax(cc.training_images, axis = 0, keepdims= True)
max_img = 0
max_img_sum = 0

cc.training_images = cc.training_images - np.full((50000, 3072), 0.5)     #biasing, but sorta random

print(cc.training_images.shape)
cc.training_images = cc.training_images.reshape(3125, 16, 3072)
print('final training image shape:', cc.training_images.shape)
cc.training_labels = cc.training_labels.reshape(3125, 16, 10)         
print('final training labels shape', cc.training_labels.shape)
cc.testing_images = cc.testing_images.reshape(10000, 3072)


cc.testing_images = cc.testing_images - np.full((10000, 3072), 0.5)      #biasing, but sorta random
print('final testing images shape', cc.testing_images.shape)

print(cc.training_labels[23][15])

#more testing
temp = cc.training_labels[23]
print(temp[15].shape)
print(temp[15][2])

(50000, 3072)
final training image shape: (3125, 16, 3072)
final training labels shape (3125, 16, 10)
final testing images shape (10000, 3072)
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
(10,)
1.0


In [ ]:
#plt.imshow(cc.training_images[344])

In [ ]:
def main():
   
    X_train = cc.training_images 
    y_train = cc.training_labels

    X_test = cc.testing_images
    y_test = cc.testing_labels

    # print('initial weight:', cc.__weights)

    cc.train(X_train, y_train)

    # print('final weight:', cc.__weights)

    predictions = cc.predict(X_test)

    #print('actual value = ', y_test, 'predicted values = ', predictions)
    # for i in range(10000):
    #   print(predictions[i][0])

    print(accuracy_score(y_test, predictions))

In [ ]:
if __name__ == "__main__":
    main()



initial weight: [[0.06830801 0.88681279 0.7191182  ... 0.46313683 0.30543594 0.47042983]
 [0.30481692 0.73741243 0.82097561 ... 0.9280699  0.22331612 0.74346521]
 [0.92981987 0.10301471 0.06738388 ... 0.96181257 0.09533702 0.72596458]
 ...
 [0.61403889 0.29114947 0.16441738 ... 0.08786888 0.71374871 0.86955177]
 [0.9723647  0.98188309 0.77254404 ... 0.21552337 0.6104757  0.34480281]
 [0.51453121 0.4941341  0.95692895 ... 0.52002672 0.68529032 0.04059785]]


100%|██████████| 1/1 [01:36<00:00, 96.65s/it]


final weight: [[0.12606849 0.97737653 0.86487803 ... 0.45526008 0.3035546  0.43451697]
 [0.24100936 0.67656363 0.85258804 ... 0.87489099 0.21958882 0.77740749]
 [0.94815486 0.18356436 0.03243772 ... 0.86497532 0.04797849 0.65915847]
 ...
 [0.70189521 0.3289739  0.27717301 ... 0.24162489 0.80927426 0.83380171]
 [0.93825494 0.91478896 0.81196238 ... 0.15095464 0.65665892 0.59053502]
 [0.46105868 0.5245161  1.05285628 ... 0.62402408 0.79807954 0.23622379]]


100%|██████████| 10000/10000 [00:17<00:00, 585.63it/s]

0.3115
